In [1]:
import sys
sys.path.append("../../nl4opt-subtask1-baseline/baseline/")

from utils.metric import SpanF1
import json

/projects/kani-lab/user/gangwar2/.miniconda3/envs/nl4opt-ner/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_spans(dataset):
    with open(dataset) as f:
        examples = [json.loads(line) for line in f]
        
    spans_dict = {}
    
    for example in examples:
        assert len(example) == 1
        key = next(iter(example.keys()))
        spans = {}
        for span in example[key]["spans"]:
            spans[(span["start"], span["end"])] = span["label"]
        spans_dict[key] = spans
        
    return spans_dict


def compare_named_entity_spans(dataset1, dataset2):
    spans_1 = read_spans(dataset1)
    spans_2 = read_spans(dataset2)
    
    assert set(spans_1.keys()) == set(spans_2.keys()), f"Spans dict mismatch"
    
    print(f"Number of examples: {len(spans_1)}")
    
    keys = spans_1.keys()
    span_f1 = SpanF1()
    
    for key in keys:
        span_f1([spans_1[key]], [spans_2[key]])
        
    metric = span_f1.get_metric(reset = True)
    return metric["micro@F1"]


def print_problem_description(dataset, example_id):
    example = None
    with open(dataset) as f:
        for line in f:
            line = json.loads(line)
            if example_id in line:
                example = line[example_id]
                break
    assert example is not None
    
    start, end, label = [], [], {}
    for span in example["spans"]:
        start.append(span["start"])
        end.append(span["end"] - 1)
        label[span["start"]] = span["label"]
    
    output = ""
    for idx, char in enumerate(example["document"]):
        if idx in start:
            output += "\colorbox{" + label[idx].replace("_", "") + "}{\strut {\tiny \textsc{" + label[idx].lower().replace("_", "\_") + "}} "
            
        output += char
        
        if idx in end:
            output += "}"
            
    return output

## p = 0.2

In [3]:
f1 = compare_named_entity_spans("../data/train.jsonl", "../data/train_noisy_p0.2.jsonl")
round(f1, 4)

Number of examples: 713


0.8333

In [4]:
f1 = compare_named_entity_spans("../data/dev.jsonl", "../data/dev_noisy_p0.2.jsonl")
round(f1, 4)

Number of examples: 99


0.8397

In [5]:
f1 = compare_named_entity_spans("../data/test.jsonl", "../data/test_noisy_p0.2.jsonl")
round(f1, 4)

Number of examples: 289


0.8353

In [6]:
print_problem_description("../data/train_noisy_p0.2.jsonl", "-707305018")

'A berry picker must pick at least \\colorbox{LIMIT}{\\strut {\tiny \textsc{limit}} 3000} strawberries and \\colorbox{LIMIT}{\\strut {\tiny \textsc{limit}} 15000} raspberries. He visits two farms. For each \\colorbox{OBJNAME}{\\strut {\tiny \textsc{obj\\_name}} hour} at \\colorbox{VAR}{\\strut {\tiny \textsc{var}} farm 1} he spends, he can pick \\colorbox{PARAM}{\\strut {\tiny \textsc{param}} 50} strawberries and \\colorbox{PARAM}{\\strut {\tiny \textsc{param}} 300} raspberries. For each \\colorbox{OBJNAME}{\\strut {\tiny \textsc{obj\\_name}} hour} at \\colorbox{VAR}{\\strut {\tiny \textsc{var}} farm 2} he spends, he can catch \\colorbox{PARAM}{\\strut {\tiny \textsc{param}} 70} strawberries and \\colorbox{PARAM}{\\strut {\tiny \textsc{param}} 200} raspberries. How many \\colorbox{OBJNAME}{\\strut {\tiny \textsc{obj\\_name}} hours} should he spend at each farm to \\colorbox{OBJDIR}{\\strut {\tiny \textsc{obj\\_dir}} minimize} the \\colorbox{OBJNAME}{\\strut {\tiny \textsc{obj\\_name}} 

## p = 0.5

In [7]:
f1 = compare_named_entity_spans("../data/train.jsonl", "../data/train_noisy_p0.5.jsonl")
round(f1, 4)

Number of examples: 713


0.5783

In [8]:
f1 = compare_named_entity_spans("../data/dev.jsonl", "../data/dev_noisy_p0.5.jsonl")
round(f1, 4)

Number of examples: 99


0.5742

In [9]:
f1 = compare_named_entity_spans("../data/test.jsonl", "../data/test_noisy_p0.5.jsonl")
round(f1, 4)

Number of examples: 289


0.5822

In [10]:
print_problem_description("../data/train_noisy_p0.5.jsonl", "-707305018")

'A berry picker must pick \\colorbox{CONSTDIR}{\\strut {\tiny \textsc{const\\_dir}} at least} \\colorbox{LIMIT}{\\strut {\tiny \textsc{limit}} 3000} strawberries and \\colorbox{LIMIT}{\\strut {\tiny \textsc{limit}} 15000} raspberries. He visits two farms. For each hour at \\colorbox{VAR}{\\strut {\tiny \textsc{var}} farm 1} he spends, \\colorbox{PARAM}{\\strut {\tiny \textsc{param}} he can} pick \\colorbox{PARAM}{\\strut {\tiny \textsc{param}} 50} strawberries and \\colorbox{PARAM}{\\strut {\tiny \textsc{param}} 300} raspberries. For each hour at farm 2 he spends, he can catch 70 strawberries and \\colorbox{PARAM}{\\strut {\tiny \textsc{param}} 200} raspberries. How many \\colorbox{OBJDIR}{\\strut {\tiny \textsc{obj\\_dir}} hours} should he spend at each farm to \\colorbox{OBJDIR}{\\strut {\tiny \textsc{obj\\_dir}} minimize} the \\colorbox{OBJNAME}{\\strut {\tiny \textsc{obj\\_name}} amount of time} he spends at both farms?'

## Predicted NER (One Epoch)

In [11]:
f1 = compare_named_entity_spans("../data/train.jsonl", "../data/train_predicted_ner.jsonl")
round(f1, 4)

Number of examples: 713


0.6713

In [12]:
f1 = compare_named_entity_spans("../data/dev.jsonl", "../data/dev_predicted_ner.jsonl")
round(f1, 4)

Number of examples: 99


0.5931

In [13]:
f1 = compare_named_entity_spans("../data/test.jsonl", "../data/test_predicted_ner.jsonl")
round(f1, 4)

Number of examples: 289


0.5961

## Predicted NER (Best)

In [14]:
f1 = compare_named_entity_spans("../data/train.jsonl", "../data/train_predicted_ner_best.jsonl")
round(f1, 4)

Number of examples: 713


0.9886

In [15]:
f1 = compare_named_entity_spans("../data/dev.jsonl", "../data/dev_predicted_ner_best.jsonl")
round(f1, 4)

Number of examples: 99


0.8934

In [16]:
f1 = compare_named_entity_spans("../data/test.jsonl", "../data/test_predicted_ner_best.jsonl")
round(f1, 4)

Number of examples: 289


0.914